In [5]:
#######################
# Accept search term from user and download
# the last 100 tweets from that search term
#######################

# Install the python-twitter module. Provides python like interface to the Twitter API. 
# Twitter API is a REST API, it provides information in the form of a JSON which
# will need to be parsed- python-twitter will do this.

import twitter

# Call pydoc twitter.api at the anaconda terminal to access documentation. 

# Twitter API keys are needed to access tweets - both from feature vector and 
# tweets for classification.

api = twitter.Api(consumer_key='6b9ebwaNU4DDa9G9xF1FhrZQt',
                 consumer_secret='2prgddykMj2b7b9zTeN78BBhdrgdaNxjtSyyoo8iNRzKAZhzMX',
                 access_token_key='817521154847969280-bc6J796tc0cRjlhigiRZIoQVIzeW2Hf',
                 access_token_secret='WAZy2gZ9Ok8NdP3W8TOMNliSUUGrLjesudvqA3nEEh9wH')

print api.VerifyCredentials()

{"created_at": "Sat Jan 07 00:00:06 +0000 2017", "default_profile": true, "default_profile_image": true, "friends_count": 101, "id": 817521154847969280, "id_str": "817521154847969280", "lang": "en", "name": "Josh Jones", "profile_background_color": "F5F8FA", "profile_image_url": "http://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png", "profile_image_url_https": "https://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png", "profile_link_color": "1DA1F2", "profile_sidebar_border_color": "C0DEED", "profile_sidebar_fill_color": "DDEEF6", "profile_text_color": "333333", "profile_use_background_image": true, "screen_name": "Jonesy_259"}


In [13]:
# Function accepts search term and then fetches the tweets for that term
def createTestData(search_string):
    try:
        tweets_fetched=api.GetSearch(search_string, count=100)
        # This will return a list with twitter.Status objects. These include
        # text, hashtags etc of the tweets that are fetched. 
        print "Great! We fetched "+str(len(tweets_fetched))+" tweets with the term "+search_string+"!!"
        # Since these tweets don't have sentiment labels yet we will 
        # keep the label empty
        return [{"text":status.text, "label":None} for status in tweets_fetched]
    except:
        print "Sorry there was an error!"
        return None
    
search_string=input("What term are we searching for?")
testData=createTestData(search_string)
    

What term are we searching for?'@RealDonaldTrump'
Great! We fetched 100 tweets with the term @RealDonaldTrump!!


In [14]:
testData[0:20]

[{'label': None,
  'text': u'Dear @realDonaldTrump: In case you forgot, here are the leaders of the Russia probe:\n\n-Republican Wray, appointed b\u2026 https://t.co/lWpFO4qiy1'},
 {'label': None,
  'text': u'Dear @TheJusticeDept Sessions: Your new policy of separating children from parents is so cruel that even\u2026 https://t.co/ogVdT5SyY3'},
 {'label': None,
  'text': u'Hey guys Roseanne Barr blocked me so please tell me what she\u2019s saying about her good buddy @realDonaldTrump ripping\u2026 https://t.co/FQ0mlAAFeY'},
 {'label': None,
  'text': u'@ABC @ThisWeekABC Homework? @realDonaldTrump doesn\u2019t read, he is read to. Prepped, only for the PR portion. #SAD'},
 {'label': None,
  'text': u'RT @realDonaldTrump: Senator Schumer and Obama Administration let phone company ZTE flourish with no security checks. I closed it down then\u2026'},
 {'label': None,
  'text': u'RT @realDonaldTrump: On behalf of the American People, CONGRATULATIONS! We love you! https://t.co/YRlj0vRXwk'},
 

In [18]:
#######################
# Classify each of the tweets postive/negative
#######################

# Download training data, Niek Sanders' Tweet Sentiment Corpus.
# Download CSV from website, fetch text for each tweet ID

def createTrainingCorpus(corpusFile,tweetDataFile):
    import csv
    corpus=[]
    with open(corpusFile, 'rb') as csvfile:
        lineReader = csv.reader(csvfile,delimiter=',',quotechar="\"")
        for row in lineReader:
            corpus.append({"tweet_id":row[2],"label":row[1],"topic":row[0]})
    # List with dictionary for each row of CSV
    # Iterate though that list and fetch the text for each tweet_id
    
    # Twitter API rate limited to 900 tweets/15min. Using delay to avoid being rate limited.
    import time
    rate_limit=900
    sleep_time=900/rate_limit
    
    trainingData=[]
    for tweet in corpus:
        try:
            status=api.GetStatus(tweet["tweet_id"])
            print "Tweet fetched" + status.text
            tweet["tweet"]=status.text
            # Tweet is a dict which already has tweet ID and label. Add tweet text.
            trainingData.append(tweet)
            time.sleep(sleep_time)
        except:
            continue
    # Store downloaded tweets in CSV
    with open(tweetDataFile,'wb') as csvfile:
        linewriter=csv.writer(csvfile,delimiter=',',quotechar="\"")
        for tweet in trainingData:
            linewriter.writerow([tweet["tweet_id"],tweet["text"],tweet["label"],tweet["topic"]])
    return trainingData

In [23]:
# Download limited Training Data to avoid time wait.

def createLimitedTrainingCorpus(corpusFile,tweetDataFile):
    import csv
    corpus=[]
    with open(corpusFile, 'rb') as csvfile:
        lineReader = csv.reader(csvfile,delimiter=',',quotechar="\"")
        for row in lineReader:
            corpus.append({"tweet_id":row[2],"label":row[1],"topic":row[0]})
    # List with dictionary for each row of CSV
    # Iterate though that list and fetch the text for each tweet_id
    
    trainingData=[]
    for label in ["positive","negative"]:
        i=1
        for tweet in corpus:
            if tweet["label"]==label and i<=50:
                try:
                    status=api.GetStatus(tweet["tweet_id"])
                    print "Tweet fetched" + status.text
                    tweet["text"]=status.text
                    # Tweet is a dict which already has tweet ID and label. Add tweet text.
                    trainingData.append(tweet)
                    i=i+1
                except Exception, e:
                    print e
                    
    # Store downloaded tweets in CSV
    with open(tweetDataFile,'wb') as csvfile:
        linewriter=csv.writer(csvfile,delimiter=',',quotechar="\"")
        for tweet in trainingData:
            try: 
                linewriter.writerow([tweet["tweet_id"],tweet["text"],tweet["label"],tweet["topic"]])
            except Exception,e:
                print e
    return trainingData


corpusFile="/Users/Josh/Downloads/twitter_corpus-master/corpus.csv"
tweetDataFile="/Users/Josh/Downloads/twitter_corpus-master/tweetDataFile.csv"

    
trainingData=createLimitedTrainingCorpus(corpusFile,tweetDataFile)
# Call function + return list with all tweet data needed for training.

Tweet fetchedNow all @Apple has to do is get swype on the iphone and it will be crack. Iphone that is
[{u'message': u'Sorry, you are not authorized to see this status.', u'code': 179}]
Tweet fetchedHilarious @youtube video - guy does a duet with @apple 's Siri. Pretty much sums up the love affair! http://t.co/8ExbnQjY
Tweet fetched@RIM you made it too easy for me to switch to @Apple iPhone. See ya!
[{u'message': u'No status found with that ID.', u'code': 144}]
[{u'message': u'No status found with that ID.', u'code': 144}]
Tweet fetchedThe 16 strangest things Siri has said so far. I am SOOO glad that @Apple gave Siri a sense of humor! http://t.co/TWAeUDBp via @HappyPlace
Tweet fetchedGreat up close & personal event @Apple tonight in Regent St store!
Tweet fetchedFrom which companies do you experience the best customer service aside from @zappos and @apple?
Tweet fetchedJust apply for a job at @Apple, hope they call me lol
[{u'message': u'No status found with that ID.', u'code': 144}]
Tw

Tweet fetchedThe one #iphone feature still missing since @apple first showed it.. Contacts pictures on the contacts list! Simple yet 5 major updates miss
Tweet fetchedAsked siri is she dreams of electric sleep. Was disappointed that she didn't have a snippy answer.  Missed opportunity @apple
[{u'message': u'Sorry, you are not authorized to see this status.', u'code': 179}]
[{u'message': u'No status found with that ID.', u'code': 144}]
Tweet fetchedFUCK YOU @apple DIE IN A FUCKING BLAZE INFERNO.
Tweet fetchedOh, @apple. Steve obviously had nothing to do with iPhoto, as it's the perfect opposite of 'insanely great'. Get it fixed, please.
[{u'message': u'Sorry, you are not authorized to see this status.', u'code': 179}]
[{u'message': u'Sorry, you are not authorized to see this status.', u'code': 179}]
[{u'message': u'Sorry, you are not authorized to see this status.', u'code': 179}]
[{u'message': u'Sorry, you are not authorized to see this status.', u'code': 179}]
Tweet fetchedshit, shit,

In [35]:
# Reproccess all tweets
import re
from nltk.tokenize import word_tokenize
from string import punctuation
from nltk.corpus import stopwords

class PreProcessTweets:
    def __init__(self):
        self._stopwords=set(stopwords.words('english')+list(punctuation)+['AT_USER','URL', 'CRP', "'s"])
        
    def processTweets(self, list_of_tweets):
        # The list of tweets is a list of disctionaries which has the keys, "text" and "label"
        processedTweets=[]
        # Each tuple is a list of words + label.
        for tweet in list_of_tweets:
            processedTweets.append((self._processTweet(tweet["text"]),tweet["label"]))
        return processedTweets
    
    def _processTweet(self, tweet):
        # Convert to lowercase
        tweet=tweet.lower()
        # Replace links with the word URL (added to stopwords -> will be removed)
        tweet=re.sub('((www\.[^\s]+) | (https?://[^\s]+))','URL',tweet)
        # Replace @username with "AT_USER (again, stopwords)
        tweet=re.sub('@[^\s]+','AT_USER',tweet)
        # Replace #word with word
        tweet=re.sub(r'#([^\s]+)',r'\1',tweet)
        # Remove numbers
        tweet=re.sub('[0-9]+','CRP',tweet)
        # Remove Unicode
        #tweet =re.sub(r'\u[^\s]+','CRP',tweet)
        
        tweet=word_tokenize(tweet)
        # Converts tweet to list of words
        # Remove stopwords
        return [word for word in tweet if word not in self._stopwords]

tweetProcessor=PreProcessTweets()
ppTrainingData=tweetProcessor.processTweets(trainingData)
ppTestData=tweetProcessor.processTweets(testData)   

In [ ]:
# Extract features and train classifier, two methods: Naive Bayes + SVM



In [ ]:
# Run the classifier on downloaded tweets

In [36]:
ppTrainingData[:20]

[([u'get', u'swype', u'iphone', u'crack', u'iphone'], 'positive'),
 ([u'hilarious',
   u'video',
   u'guy',
   u'duet',
   u'siri',
   u'pretty',
   u'much',
   u'sums',
   u'love',
   u'affair'],
  'positive'),
 ([u'made', u'easy', u'switch', u'iphone', u'see', u'ya'], 'positive'),
 ([u'strangest',
   u'things',
   u'siri',
   u'said',
   u'far',
   u'sooo',
   u'glad',
   u'gave',
   u'siri',
   u'sense',
   u'humor',
   u'via'],
  'positive'),
 ([u'great',
   u'close',
   u'personal',
   u'event',
   u'tonight',
   u'regent',
   u'st',
   u'store'],
  'positive'),
 ([u'companies', u'experience', u'best', u'customer', u'service', u'aside'],
  'positive'),
 ([u'apply', u'job', u'hope', u'call', u'lol'], 'positive'),
 ([u'lmao',
   u'think',
   u'onto',
   u'something',
   u'magical',
   u'dying',
   u'haha',
   u'siri',
   u'suggested',
   u'find',
   u'whores',
   u'hide',
   u'body',
   u'lolol'],
  'positive'),
 ([u'rt',
   u'registered',
   u'developer',
   u'...',
   u'hoping',
 